## Large File Visualization and Splitting

### Visualization with Napari

use img.set_scene to set what number of scene for selection - the number starts with 0

In [ ]:

##run in napari env - customize
from aicsimageio import AICSImage
from aicsimageio.writers import OmeTiffWriter
# Get an AICSImage object
img = AICSImage("Exp67_20230504_UOK121_CENPA_568_TEAD1_488_Hoeschet_01_Z-stack_01-Airyscan Processing-15.czi")
img.data  # returns 6D STCZYX numpy array
img.dims  # returns string "STCZYX"
img.shape  # returns tuple of dimension sizes in STCZYX order
print("Dimentions will be", img.dims)
print("There are", len(img.scenes), "of scenes")
#Here you set what number of scene for selection - the number starts with 0
img.set_scene(0)
first_channel_data = img.get_image_data("ZYX")

### Add single channel image to Napari


In [ ]:
import napari
viewer = napari.view_image(first_channel_data, colormap='magma')

#add second position to the napari GUI
img.set_scene(0)
data_4 = img.get_image_data("ZYX")
new_layer = viewer.add_image(data_4, colormap='magma')

### Add multi-channel Image to Napari

In [5]:
# for multi-channel images
import napari
viewer = napari.Viewer()
channel_names = img.channel_names
img.set_scene(0)
with napari.gui_qt():
    viewer = napari.Viewer()
    for c, c_name in enumerate(channel_names):
        viewer.add_image(img.get_image_data("ZYX", C=c, S=0, T=0), 
                         name=c_name)

/opt/homebrew/anaconda3/envs/napari/lib/python3.9/contextlib.py:119: FutureWarning: 
The 'gui_qt()' context manager is deprecated.
If you are running napari from a script, please use 'napari.run()' as follows:

    import napari

    viewer = napari.Viewer()  # no prior setup needed
    # other code using the viewer...
    napari.run()

In IPython or Jupyter, 'napari.run()' is not necessary. napari will automatically
start an interactive event loop for you: 

    import napari
    viewer = napari.Viewer()  # that's it!

  return next(self.gen)


### Split Image 

First check with file list

In [4]:
from pathlib import Path

mydir = Path("airyscan/")
file_list = [] # create an empty list

for file in mydir.iterdir():
    if file.suffix == '.czi' and not file.name.startswith("._"):
        file_list.append(str(file)) # append in files in folder with .tif

file_list

['airyscan/Exp67_20230504_UOK121_CENPA_568_TEAD1_488_Hoeschet_01_Z-stack_01-Airyscan Processing-15.czi',
 'airyscan/Exp67_20230504_UOK121_CENPA_568_TEAD1_488_Hoeschet_01_Z-stack_02-Airyscan Processing-16.czi',
 'airyscan/Exp67_20230504_UOK121_CENPA_568_TEAD1_488_Hoeschet_01_Z-stack_03-Airyscan Processing-17.czi',
 'airyscan/Exp67_20230504_UOK121_CENPA_568_TEAD1_488_Hoeschet_01_Z-stack_04-Airyscan Processing-18.czi',
 'airyscan/Exp67_20230504_UOK121_CENPA_568_TEAD1_488_Hoeschet_01_Z-stack_05-Airyscan Processing-19.czi',
 'airyscan/Exp67_20230504_UOK121_CENPA_568_TEAD1_488_Hoeschet_01_Z-stack_06-Airyscan Processing-21.czi']

### Split Image

Split only 'scenes/position'.

In [5]:
import os
from aicsimageio import AICSImage
from aicsimageio.writers import OmeTiffWriter
from pathlib import Path

#following code only change when 3 channals

if not os.path.exists("images"):
    os.makedirs("images")
for img in file_list: 
    filename = Path(img).stem
    img2 = AICSImage(img) # Open an czi image in file_list
    img_channels = img2.dims.C
    img_scenes = len(img2.scenes)
    img_time = img2.dims.T
    # print(img_channels,img_scenes,img_time)
    for S in range(img_scenes): #loop through scene by scene (aka positions)
        # for T in range(img_time): #loop through time by time
            # for C in range(img_channels):
                img2.set_scene(S)
                # print(S,T,C)
                #single_img_data = img2.get_image_data("ZYX",S=S,T=T,C=C)
                single_img_data = img2.get_image_data("TCZYX",S=S)
 #for two channel image, split C1C2#for three channel image, split C1C2C3#for four channel image, spliot C1C2C3C4
                if img_channels == 2:
                    file_name = f"{filename}_{os.path.splitext(img)[1]}.S{S+1:03d}.T{T+1:03d}.C{C+1:03d}.ome.tif"
                    OmeTiffWriter.save(single_img_data,f"./images/{file_name}")
                if img_channels == 3:
                    file_name = f"{filename}_{os.path.splitext(img)[1]}.S{S+1:03d}.ome.tif"
                    OmeTiffWriter.save(single_img_data,f"./images/{file_name}")
                elif img_channels == 4:
                    file_name = f"{filename}_{os.path.splitext(img)[1]}.S{S+1:03d}.T{T+1:03d}.C{C+1:03d}.ome.tif"
                    OmeTiffWriter.save(single_img_data,f"./images/{file_name}")               